In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens100k/movies.csv
/kaggle/input/movielens100k/ratings.csv
/kaggle/input/movielens100k/u.data
/kaggle/input/movielens100k/tags.csv
/kaggle/input/movielens100k/links.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')
  

In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=01ffdad4cf66fce55d431eb1ac0b0da8474c509f50e069a8d05f87d1dfc6db56
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession \
    .builder \
    .appName("Movie_Recommender") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/04 00:35:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# read ratings csv file
ratings = spark.read.csv("/kaggle/input/movielens100k/ratings.csv",header = True)
# Look at the column names
print(ratings.columns)

# Look at the first few rows of data
print(ratings.show())

['userId', 'movieId', 'rating', 'timestamp']
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows

None


In [7]:
# Count the total number of ratings in the dataset
total_count = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
users_count = ratings.select("userId").distinct().count()
movies_count = ratings.select("movieId").distinct().count()

print("number of Users in dataset are",users_count)
print("number of movies in dataset are",movies_count)

number of Users in dataset are 671
number of movies in dataset are 9066


In [8]:
# Set the denominator equal to the number of users multiplied by the number of movies
total_possibilities = users_count*movies_count
ratings_filled = (total_count/total_possibilities)*100
print("Total Percentage Ratings acutally given in Data",round(ratings_filled,2),"%")
print("Total Percentage Ratings need to be found",100 - round(ratings_filled,2), "%")

Total Percentage Ratings acutally given in Data 1.64 %
Total Percentage Ratings need to be found 98.36 %


In [9]:
# Import the requisite packages
from pyspark.sql.functions import col

# View the ratings dataset
ratings.show(10)


+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
+------+-------+------+----------+
only showing top 10 rows



In [10]:
# Highest number of movies rated by users
ratings.groupBy("userId").count().sort("count",ascending = False).show(10)

+------+-----+
|userId|count|
+------+-----+
|   547| 2391|
|   564| 1868|
|   624| 1735|
|    15| 1700|
|    73| 1610|
|   452| 1340|
|   468| 1291|
|   380| 1063|
|   311| 1019|
|    30| 1011|
+------+-----+
only showing top 10 rows



In [11]:
from pyspark.sql.functions import min,avg
# Min num ratings for movies
print("Minimum ratings for movies")
ratings.groupBy("movieId").count().select(min("count")).show(10)

# Avg num ratings per movie
print("Avg number of ratings per movie: ")
ratings.groupBy("movieId").count().select(avg("count")).show(10)


Minimum ratings for movies
+----------+
|min(count)|
+----------+
|         1|
+----------+

Avg number of ratings per movie: 
+------------------+
|        avg(count)|
+------------------+
|11.030664019413193|
+------------------+



In [12]:
# Min num ratings for user
print("User with the Least number of ratings: ")
ratings.groupBy("userId").count().select(min("count")).show(10)

# Avg num ratings per users
print("Avg number of ratings per user: ")
ratings.groupBy('userId').count().select(avg("count")).show(10)

User with the Least number of ratings: 
+----------+
|min(count)|
+----------+
|        20|
+----------+

Avg number of ratings per user: 
+------------------+
|        avg(count)|
+------------------+
|149.03725782414307|
+------------------+



In [13]:
# Use .printSchema() to see the datatypes of the ratings dataset
ratings.printSchema()


root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [14]:

# Convertign userId and movieid to Integers
ratings = ratings.select(ratings.userId.cast("integer"), ratings.movieId.cast("integer"), ratings.rating.cast("double"))

# Call .printSchema() again to confirm the columns are now in the correct format
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [15]:
# Split the ratings dataframe into training and test data
(training_data, test_data) = ratings.randomSplit([0.8,0.2], seed=123)
training_data.show(10)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1129|   2.0|
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1293|   2.0|
|     1|   1339|   3.5|
|     1|   1343|   2.0|
|     1|   1371|   2.5|
|     1|   1405|   1.0|
+------+-------+------+
only showing top 10 rows



In [16]:

# Set the ALS hyperparameters
from pyspark.ml.recommendation import ALS
recommender_model = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", 
          rank =10, maxIter =15, regParam =0.1,
          coldStartStrategy="drop", nonnegative =True, 
          implicitPrefs = False)
print(recommender_model)

ALS_265f2cee3e3d


In [17]:
import time
start_time = time.time()
# Fit the model to the training_data
model = recommender_model.fit(training_data)
end_time = time.time()

execution_time = round(end_time - start_time,2)
print("Execution time {} seconds:".format(execution_time))


Execution time 12.52 seconds:


In [18]:
import time
start_time = time.time()

# Generate predictions on the test_data
test_predictions = model.transform(test_data)

end_time = time.time()
execution_time = round(end_time - start_time,2)
print("Total time to predict {} seconds:".format(execution_time))


Total time to predict 0.14 seconds:


In [19]:
test_predictions.show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|     32|   4.0| 4.0136228|
|   148|    480|   4.5| 3.9385648|
|   148|    492|   3.5|   3.72876|
|   148|   1022|   4.5| 3.8027527|
|   148|   1097|   5.0|  4.002677|
|   148|   1234|   4.0|  4.203488|
|   148|   1252|   5.0| 4.3973627|
|   148|   1284|   4.0| 3.9353528|
|   148|   1288|   4.5| 4.2435026|
|   148|   1449|   4.5| 4.2691035|
+------+-------+------+----------+
only showing top 10 rows



In [20]:
from pyspark.ml.evaluation import RegressionEvaluator

# Using rating as given rating and prediction as predicted rating
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="rating", 
                                predictionCol="prediction")

In [21]:
# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print ("RMSE value is",RMSE)

RMSE value is 0.9014408612419802


**An RMSE of 0.9 means that on average the model predicts 0.9 above or below values of the original ratings matrix**

In [22]:
# Look at user 60's ratings
print("User 60's Ratings:")
ratings.filter(col("userId") == 60).sort("rating", ascending = False).show(10)

User 60's Ratings:
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    60|   2324|   5.0|
|    60|  30749|   5.0|
|    60|   3949|   5.0|
|    60|    235|   5.0|
|    60|   5060|   5.0|
|    60|   1732|   5.0|
|    60|   5690|   5.0|
|    60|   1653|   5.0|
|    60|   5995|   5.0|
|    60|   7361|   5.0|
+------+-------+------+
only showing top 10 rows



In [23]:

# Look at the movies recommended to user 60
print("User 60s Recommendations:")
test_predictions.filter(col("userId") == 60).show(10)

User 60s Recommendations:


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    60|   8638|   5.0| 4.2529488|
|    60|    858|   5.0| 4.7113037|
|    60|  30749|   5.0|  4.348521|
|    60|   6350|   5.0|  4.287865|
|    60|   4973|   4.5| 4.5075116|
|    60|   5959|   4.5| 3.7624574|
|    60|   1732|   5.0| 4.6733828|
|    60|    720|   4.0| 4.1327705|
|    60|   3608|   4.0| 3.3798032|
|    60|   1209|   4.0| 4.4556427|
+------+-------+------+----------+
only showing top 10 rows



In [24]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Create test and train set
(train, test) = ratings.randomSplit([0.8,0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False)

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [25]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
           .addGrid(als.rank, [10, 50]) \
           .addGrid(als.maxIter, [5, 50, 100]) \
           .addGrid(als.regParam, [.05, .1]) \
           .build()

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  12


In [26]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_6cd55b4ba020


In [27]:
spark.sparkContext.setCheckpointDir("./checkpoint")
train = train.checkpoint()
train = train.cache()

In [28]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

In [29]:

#Extract best model from the cv model above
best_model = model.bestModel
best_model

ALSModel: uid=ALS_fa21b69b647f, rank=10

In [30]:
test_predictions = best_model.transform(test)
# View the predictions 
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|   1029|   3.0| 1.3357784|
|     1|   1172|   4.0| 2.7076385|
|     2|     17|   5.0|  3.921152|
|     2|     50|   4.0| 4.0954256|
|     2|     52|   3.0| 3.0796947|
|     2|     62|   3.0| 3.7658153|
|     2|    110|   4.0| 3.3828373|
|     2|    208|   3.0| 2.4410667|
|     2|    266|   5.0| 3.9223788|
|     2|    273|   4.0| 2.9463706|
|     2|    317|   2.0| 2.6824007|
|     2|    350|   4.0| 3.6470108|
|     2|    356|   3.0| 3.4065514|
|     2|    357|   3.0|  4.122164|
|     2|    372|   3.0| 3.5786264|
|     2|    405|   2.0| 1.6343349|
|     2|    474|   2.0| 4.3215013|
|     2|    480|   4.0| 3.4869316|
|     2|    508|   4.0| 3.2728367|
|     2|    552|   3.0|  3.373544|
+------+-------+------+----------+
only showing top 20 rows



In [31]:
test_predictions.count()

19855

In [32]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     2|    350|   4.0| 3.6470108|
|     2|    372|   3.0| 3.5786264|
|     1|   1172|   4.0| 2.7076385|
|     2|     52|   3.0| 3.0796947|
|     2|    474|   2.0| 4.3215013|
|     1|   1029|   3.0| 1.3357784|
|     2|    266|   5.0| 3.9223788|
|     2|     17|   5.0|  3.921152|
|     2|    552|   3.0|  3.373544|
|     2|    586|   3.0| 2.3760915|
|     2|    273|   4.0| 2.9463706|
|     2|     50|   4.0| 4.0954256|
|     2|    356|   3.0| 3.4065514|
|     2|     62|   3.0| 3.7658153|
|     2|    480|   4.0| 3.4869316|
|     2|    110|   4.0| 3.3828373|
|     2|    508|   4.0| 3.2728367|
|     2|    317|   2.0| 2.6824007|
|     2|    357|   3.0|  4.122164|
|     2|    405|   2.0| 1.6343349|
+------+-------+------+----------+
only showing top 20 rows



In [33]:
predictions_notNull = test_predictions.filter(test_predictions.prediction.isNotNull())
predictions_notNull.show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|   1029|   3.0| 1.3357784|
|     1|   1172|   4.0| 2.7076385|
|     2|     17|   5.0|  3.921152|
|     2|     50|   4.0| 4.0954256|
|     2|     52|   3.0| 3.0796947|
|     2|     62|   3.0| 3.7658153|
|     2|    110|   4.0| 3.3828373|
|     2|    208|   3.0| 2.4410667|
|     2|    266|   5.0| 3.9223788|
|     2|    273|   4.0| 2.9463706|
+------+-------+------+----------+
only showing top 10 rows



In [34]:
predictions_notNull.count()

19855

In [35]:
test_pd = test_predictions.toPandas()
test_pd.head()

,userId,movieId,rating,prediction
0,148,58,4.0,3.998277
1,148,329,4.0,2.967069
2,148,596,4.5,3.922813
3,148,903,5.0,4.144999
4,148,913,4.0,4.553446


In [36]:
test_pd.isnull().sum()

userId          0
movieId         0
rating          0
prediction    687
dtype: int64

In [37]:
test2 = test_pd.dropna()
test2.isnull().sum()

userId        0
movieId       0
rating        0
prediction    0
dtype: int64

In [38]:
test2.head()

,userId,movieId,rating,prediction
0,148,58,4.0,3.998277
1,148,329,4.0,2.967069
2,148,596,4.5,3.922813
3,148,903,5.0,4.144999
4,148,913,4.0,4.553446


In [39]:
test2["error"] = test2.rating - test2.prediction
test2.head()

,userId,movieId,rating,prediction,error
0,148,58,4.0,3.998277,0.001723
1,148,329,4.0,2.967069,1.032931
2,148,596,4.5,3.922813,0.577187
3,148,903,5.0,4.144999,0.855001
4,148,913,4.0,4.553446,-0.553446


In [40]:
test2.error.describe()

count    19168.000000
mean         0.103396
std          0.931421
min         -4.299721
25%         -0.420337
50%          0.142838
75%          0.690116
max          4.097555
Name: error, dtype: float64

**RMSE score based on best hyper parameters**

In [47]:
rmse = round(test2.error.mean(),4)
print("RMSE score using best hyperparameters",rmse)

RMSE score using best hyperparameters 0.1034


**An RMSE of 0.1 means that on average the model predicts 0.1
above or below values of the original ratings matrix**

**Top 5 movies for User 148**

In [41]:
test2[test2.userId == 148].sort_values(by = ["prediction"],ascending = False).head()

,userId,movieId,rating,prediction,error
4,148,913,4.0,4.553446,-0.553446
14,148,2087,4.0,4.490499,-0.490499
26,148,6561,4.5,4.394219,0.105781
24,148,5952,4.5,4.393504,0.106496
20,148,4226,5.0,4.332656,0.667344


In [42]:
movies = spark.read.csv("/kaggle/input/movielens100k/movies.csv",header = True)
movies.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



**Return top 10 movies based on user Id**

In [43]:
def recommend_top_10_movies(userId):
    user_Id = ratings.filter(ratings.userId == userId).select(ratings.userId)
    userSubsetRecs = best_model.recommendForUserSubset(user_Id, 10)
    movie_rating = userSubsetRecs.filter(userSubsetRecs.userId == 148)\
    .select(userSubsetRecs.recommendations).toPandas()
    movies_list = [] 
    for v in movie_rating.recommendations:
        for i in range(10):
            movies_list.append(v[i].__getitem__("movieId"))
    movie_titles = movies.filter(movies.movieId.isin(movies_list)).select("title")\
    .show(10,truncate = False)

**Top 10 movies for user148**

In [44]:
recommend_top_10_movies(148)

+----------------------------------------------------------------------------+
|title                                                                       |
+----------------------------------------------------------------------------+
|Cyclo (Xich lo) (1995)                                                      |
|Wings of Desire (Himmel über Berlin, Der) (1987)                            |
|Hands on a Hard Body (1996)                                                 |
|Bad and the Beautiful, The (1952)                                           |
|Land of Silence and Darkness (Land des Schweigens und der Dunkelheit) (1971)|
|Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's Tale) (2009)                  |
|Goat, The (1921)                                                            |
|Cops (1922)                                                                 |
|The Artist (2011)                                                           |
|Fawlty Towers (1975-1979)                          